# [Day 16](https://adventofcode.com/2024/day/16)

## Part 1

In [154]:
from typing import List, Tuple
from heapq import heappop, heappush

class Maze:

    def __init__(self, file_name: str):
        self.file_name = file_name
        self.maze = self.load_maze(file_name)
        self.start_pos = self.maze["S"]  # start position
        self.start_orientation = ">"  # start orientation
        self.end_pos = self.maze["E"]  # end position
        self.wall_pos = self.maze["#"]  # wall positions
        self.steps = {"<": (-1, 0), ">": (1, 0), "^": (0, -1), "v": (0, 1)}

    def __repr__(self) -> str:
        return self.print_maze()

    @staticmethod  
    def load_maze(file_name: str) -> dict:
        with open(file_name, "r") as f:
            puzzle_input = f.read().splitlines()
            maze = {}
            for y, line in enumerate(puzzle_input):
                for x, char in enumerate(line):
                    if char in ["S", "E"]:
                        maze.setdefault(char, (x,y))
                    if char in ["#"]:
                        maze.setdefault(char, [])
                        maze[char].append((x,y))
        return maze

    def solve_maze(self):
        # Implement a depth first search algorithm

        # Priority queue for Dijkstra's
        pq = []
        heappush(pq, (0, self.start_pos, self.start_orientation))

        visited = set()
        came_from = {}  # To track the path: pos -> previous_pos
        # prev_pos = None

        while pq:
            cost, pos, orientation = heappop(pq)

            if (pos, orientation) in visited:
                continue
            visited.add((pos, orientation))

            # Track where we came from for path reconstruction
            if pos not in came_from:  # Avoid overwriting positions with orientations
                came_from[pos] = None if pos == self.start_pos else prev_pos

            # Check if we've reached the end position
            if pos == self.end_pos:
                optimal_path = self.reconstruct_path(came_from)
                return cost, optimal_path

            # Explore neighbours
            for new_pos, new_orientation, action_cost in self.get_next_states(pos, orientation):
                if (new_pos, new_orientation) not in visited:
                    heappush(pq, (cost + action_cost, new_pos, new_orientation))
                    prev_pos = pos  # Set previous position for the neighbor

        return -1, None  # No path found

    def get_next_states(self, pos: tuple, orientation: str):
        next_states = []

        # Forward move (cost 1)
        x, y = pos
        dx, dy = self.steps[orientation]
        new_pos = x + dx, y + dy

        if new_pos not in self.wall_pos:  # Ensure it's a valid move
            next_states.append((new_pos, orientation, 1))  # Position, orientation, cost

        directions = ["<", "^", ">", "v"]
        for turn, cost in [(-1, 1000), (1, 1000)]:
            i = (directions.index(orientation) + turn) % 4
            new_orientation = directions[i]
            next_states.append((pos, new_orientation, cost))

        return next_states

    def reconstruct_path(self, came_from: dict):
        """Reconstruct the path based on positions only."""
        path = []
        current = self.end_pos
        while current:
            path.append(current)
            current = came_from[current]
        return path  # Reverse to get the path from start to end

    def get_position(self, move_str: str):
        pos = (0, 0)

        # Keep track of the previous move
        prev_move = move_str[0]  # Start move

        for move in move_str[1:]:
            if move == prev_move:  # Direction didn't changed; take a step
                step = self.steps[move]  # Get the step for the current move
                pos = (pos[0] + step[0], pos[1] + step[1])  # Update position
            prev_move = move  # Update previous move
        return pos

    def print_maze(self):
        # Write the grid to a text file
        with open(self.file_name, "r") as f:
            maze = f.read()
        return maze

    def print_maze_positions(self, positions: List[Tuple[int, int]]):
        maze = self.print_maze().splitlines()
        for y, line in enumerate(maze):
            for x, char in enumerate(line):
                if (x, y) in positions:
                    maze[y] = maze[y][:x] + "O" + maze[y][(x+1):]
        maze_str = ""            
        for line in maze:
            maze_str = maze_str + line + "\n"
        return print(maze_str)

    def print_maze_with_path(self, optimal_path):
        # Determine the size of the maze
        all_positions = [self.maze["S"], self.maze["E"]] + list(self.maze["#"])
        max_x = max(pos[0] for pos in all_positions) + 1
        max_y = max(pos[1] for pos in all_positions) + 1

        # Create an empty grid
        grid = [[" " for _ in range(max_x)] for _ in range(max_y)]

        # Mark walls
        #for x, y in self.maze["#"]:
            #grid[y][x] = "#"

        # Mark the start and end positions
        start_x, start_y = self.maze["S"]
        end_x, end_y = self.maze["E"]
        grid[start_y][start_x] = "S"
        grid[end_y][end_x] = "E"

        # Mark the path
        for x, y in optimal_path:
            #if grid[y][x] == " ":  # Avoid overwriting start or end
            grid[y][x] = "O"

        # Print the grid
        for row in grid:
            print("".join(row))

maze = Maze("data.txt")
score, optimal_paths = maze.solve_maze()
print(f"Score is {score}")

Score is 11048


In [155]:
maze.print_maze_positions(optimal_paths)

#################
#...#...#...#O.O#
#.#.#.#.#.#.#O#O#
#O#.#.#...#..O#O#
#O#O#.#.###.#.#O#
#O.O#.#.#OOOO.#O#
#O#.#.#.#O#####.#
#O#...#.#.#OOOO.#
#O#.#####.#O###O#
#O#.#.....OO#OOO#
#O#.###.#####O###
#O#.#OOO#OOOOO#.#
#O#O#.#####.###.#
#O#O#..OOOOOO.#.#
#O#O#.#########.#
#O#OOOOOO.......#
#################



In [156]:
maze.print_maze_with_path(optimal_paths)

                 
             O O 
             O O 
 O           O O 
 O O           O 
 O O     OOOO  O 
 O       O       
 O         OOOO  
 O         O   O 
 O        OO OOO 
 O           O   
 O   OOO OOOOO   
 O O             
 O O   OOOOOO    
 O O             
 O OOOOOO        
                 
